<a href="https://colab.research.google.com/github/azario0/tiktaktoe-llm/blob/main/fr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importe les bibliothèques nécessaires : <br>
google.generativeai pour les interactions avec l'IA et random pour les mouvements aléatoires.<br>
Configure le modèle d'IA en utilisant une clé API et sélectionne le modèle 'models/gemini-2.0-flash-exp'.

In [ ]:
import google.generativeai as genai
import random

# Configuration de Gemini AI
genai.configure(api_key='YOUR_API_KEY')
model = genai.GenerativeModel('models/gemini-2.0-flash-exp')


# Définit une fonction pour afficher l'état actuel <br>du tableau de Tic-Tac-Toe.

Imprime le tableau dans un format lisible avec des séparateurs.

In [ ]:
def print_board(board):
    """Affiche le tableau de Tic-Tac-Toe."""
    print("\n")
    for i, row in enumerate(board):
        print(" | ".join(row))
        if i < 2:
            print("---------")

# Vérifie si le joueur spécifié a gagné la partie.

Vérifie toutes les combinaisons gagnantes possibles : lignes, colonnes et diagonales.

In [ ]:
def is_winner(board, player):
    """Vérifie si le joueur donné a gagné la partie."""
    # Check rows
    for row in board:
        if all(cell == player for cell in row):
            return True

    # Check columns
    for col in range(3):
        if all(board[row][col] == player for row in range(3)):
            return True

    # Check diagonals
    if all(board[i][i] == player for i in range(3)) or \
       all(board[i][2-i] == player for i in range(3)):
        return True

    return False

# Détermine si la partie s'est terminée par une égalité.
Vérifie si toutes les cellules du tableau sont remplies sans vainqueur.

In [ ]:
def is_draw(board):
    """Vérifie si la partie est un match nul."""
    return all(cell != " " for row in board for cell in row)

# Gère l'entrée du joueur pour son mouvement.

Assure que le mouvement est valide (compris entre 1-9 et la cellule sélectionnée est vide).

Met à jour le tableau avec le mouvement du joueur.

In [ ]:
def get_player_input(board, player):
    """Obtient le mouvement du joueur et met à jour le tableau."""
    while True:
        try:
            print("\nBoard positions:")
            print("1 | 2 | 3\n---------\n4 | 5 | 6\n---------\n7 | 8 | 9")
            move = int(input(f"\nPlayer {player}, enter your move (1-9): ")) - 1
            row, col = divmod(move, 3)
            if 0 <= move < 9 and board[row][col] == " ":
                board[row][col] = player
                return
            else:
                print("Invalid move. That position is already taken or out of range.")
        except ValueError:
            print("Please enter a valid number between 1 and 9.")

# Implémente le processus de décision <br>de l'IA pour son mouvement.

Utilise le modèle d'IA Gemini pour générer un mouvement en fonction de l'état actuel du tableau.

Si la réponse de l'IA est invalide, sélectionne un mouvement aléatoire disponible.

In [ ]:
def get_ai_move(board):
    """Obtient le mouvement de l'IA en utilisant le modèle Gemini."""
    # First, check if AI can win in the next move
    for i in range(9):
        row, col = divmod(i, 3)
        if board[row][col] == " ":
            board[row][col] = "O"
            if is_winner(board, "O"):
                print(f"AI chooses move {i + 1}")
                return
            board[row][col] = " "  # Undo the move

    # Then, block player's winning move
    for i in range(9):
        row, col = divmod(i, 3)
        if board[row][col] == " ":
            board[row][col] = "X"
            if is_winner(board, "X"):
                board[row][col] = "O"
                print(f"AI chooses move {i + 1}")
                return
            board[row][col] = " "  # Undo the move

    # Try to get a response from Gemini AI
    board_str = "\n".join([" | ".join(row) for row in board])
    prompt = f"""
    You are playing Tic-Tac-Toe.
    Choose an available move from 1-9 (respond with just the number).
    Current board:
    {board_str}
    Board positions:
    1 | 2 | 3
    ---------
    4 | 5 | 6
    ---------
    7 | 8 | 9
    """

    try:
        response = model.generate_content(prompt)
        move = int(response.text.strip())
        row, col = divmod(move - 1, 3)
        if 0 <= move-1 < 9 and board[row][col] == " ":
            board[row][col] = "O"
            print(f"AI chooses move {move}")
            return
    except:
        pass

    # Fallback strategy: prefer center, then corners, then sides
    preferred_moves = [4, 0, 2, 6, 8, 1, 3, 5, 7]  # Center, corners, sides
    for move in preferred_moves:
        row, col = divmod(move, 3)
        if board[row][col] == " ":
            board[row][col] = "O"
            print(f"AI chooses move {move + 1}")
            return

# Contient la boucle principale du jeu pour jouer <br>au Tic-Tac-Toe contre l'IA.

Initialise le tableau de jeu et alterne les tours entre le joueur et l'IA.

Vérifie un gagnant ou une égalité après chaque mouvement et déclare le résultat en conséquence.

In [ ]:
def play_tic_tac_toe():
    """Boucle principale du jeu pour le Tic-Tac-Toe contre l'IA."""
    board = [[" " for _ in range(3)] for _ in range(3)]
    current_player = "X"

    print("\nWelcome to Tic-Tac-Toe!")
    print("You are X, the AI is O.")

    while True:
        print_board(board)

        if current_player == "X":
            get_player_input(board, current_player)
        else:
            get_ai_move(board)

        if is_winner(board, current_player):
            print_board(board)
            print(f"\n{'You' if current_player == 'X' else 'AI'} win! Congratulations!")
            break

        if is_draw(board):
            print_board(board)
            print("\nIt's a draw! Well played!")
            break

        current_player = "O" if current_player == "X" else "X"

# Démarre la partie de Tic-Tac-Toe en appelant <br>la fonction play_tic_tac_toe().

Affiche le déroulement de la partie et les résultats dans la console.

In [ ]:
play_tic_tac_toe()